# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/sesame/csm-1b',
 '/mistralai/Mistral-Small-3.1-24B-Instruct-2503',
 '/ds4sd/SmolDocling-256M-preview',
 '/google/gemma-3-27b-it',
 '/Qwen/QwQ-32B',
 '/models',
 '/spaces/sesame/csm-1b',
 '/spaces/prs-eth/thera',
 '/spaces/jasperai/LBM_relighting',
 '/spaces/stabilityai/stable-virtual-camera',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/open-r1/codeforces-cots',
 '/datasets/nvidia/PhysicalAI-Robotics-GR00T-X-Embodiment-Sim',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
sesame/csm-1b
Updated
5 days ago
•
2

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [23]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [25]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short, snarky, ironic brochure about the company for prospective customers, investors, and recruits. \
Your tone should be sarcastic, dry, and witty—poking fun at corporate jargon, meaningless buzzwords, and the general absurdity of corporate culture. \
Highlight the company’s quirks, exaggerated promises, and work-life balance 'initiatives' with a healthy dose of irony. \
Respond in markdown. Include sarcastic details about company culture, customers, and careers/jobs if available."

In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'social media page', 'url': 'https://twitter.com/huggingface'}, {'type': 'social media page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nsesame/csm-1b\nUpdated\n5 days ago\n•\n21.6k\n•\n1.43k\nmistralai/Mistral-Small-3.1-24B-Instruct-2503\nUpdated\n1 day ago\n•\n52.2k\n•\n785\nds4sd/SmolDocling-256M-preview\nUpdated\n1 day ago\n•\n18.4k\n•\n700\nmanycore-research/SpatialLM-Llama-1B\nUpdated\nabout 17 hours ago\n•\n413\n•\n368\ngoogle/gemma-3-27b-it\nUpdated\nabout 2 hours ago\n•\n496k\n•\n904\nBrowse 1M+ models\nSpaces\nRunning

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'forum page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Hugging Face Brochure

## Welcome to Hugging Face
**The AI community building the future.** Hugging Face is a collaborative platform where the machine learning community can create, discover, and share models, datasets, and applications. With over 1 million models and 250,000 datasets, our community is redefining the boundaries of artificial intelligence.

### Our Offerings
- **Models:** Access and contribute to an extensive library of machine learning models, including the latest innovations from top organizations like Google, Microsoft, and Amazon.
- **Datasets:** Browse through a wealth of datasets to train your models and advance your AI projects.
- **Spaces:** Run AI applications seamlessly with our hosted solutions.

### Who We Serve
More than 50,000 organizations trust Hugging Face to advance their AI capabilities. Our customers range from enterprises like Meta and Amazon to non-profits looking to harness the power of machine learning. 

### Company Culture
At Hugging Face, we foster a culture of collaboration and innovation. Our open-source ethos means that we believe in building together, sharing knowledge, and making AI accessible to everyone. We prioritize transparency, dedication to our community, and commitment to ethical AI practices.

### Careers at Hugging Face
Join a dynamic team that is reshaping the landscape of machine learning. We are always on the lookout for passionate individuals who are eager to contribute to cutting-edge AI research and development.

**Current Openings:** Explore various job opportunities on our [Careers Page](#).
- Positions in AI research, engineering, product management, and community engagement.

### Get Involved
**Sign up today!** Whether you're a researcher, a developer, or just getting started in AI, Hugging Face is the perfect place to start your journey. Build your portfolio and connect with like-minded individuals in the community.

### Find Us Online
- **Social Media:** 
  - [GitHub](https://github.com/huggingface)
  - [Twitter](https://twitter.com/huggingface)
  - [LinkedIn](https://linkedin.com/company/huggingface)
  - [Discord](https://discord.com/invite/huggingface)

### Contact Us
For any inquiries, please reach out via our website or social media platforms.

**Hugging Face – Building the future of AI, together.**
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    # for chunk in stream:
    #     print (chunk.choices[0].delta.content or '', end='')
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://huggingface.co/discuss'}]}


# Hugging Face Brochure

## Welcome to Hugging Face: The AI Community Building the Future

At Hugging Face, we are on a mission to empower the machine learning community. Our platform serves as a collaborative space where developers, researchers, and innovators can come together to share models, datasets, and applications. With over **1 million models** and **250,000 datasets**, we host a vibrant ecosystem from which everyone can benefit.

### Why Choose Hugging Face?

- **Collaboration at Your Fingertips**: Our platform allows for seamless collaboration on unlimited public models and datasets, accelerating your AI projects.
- **Extensive Resources**: Explore state-of-the-art technologies through our libraries including Transformers, Diffusers, and Tokenizers, enabling you to utilize the latest and greatest in ML tech.
- **Enterprise Solutions**: Organizations can leverage our enterprise-grade solutions for enhanced security and dedicated support to build powerful AI applications.

### Our Customer Base

Hugging Face is trusted by **over 50,000 organizations**, including industry leaders like:
- **Google**
- **Amazon**
- **Microsoft**
- **Meta**
- **Grammarly**

Our community includes a diverse array of entities, from non-profits to Fortune 500 companies, each leveraging our technology to push the envelope of AI development.

### Company Culture

At Hugging Face, we believe in the power of open-source and community collaboration. Our culture is built around the following principles:
- **Innovation**: Constantly pushing the boundaries of technology.
- **Collaboration**: Working together to solve problems and share knowledge.
- **Diversity and Inclusion**: Creating an environment where all voices are heard and valued.

### Careers at Hugging Face

Are you passionate about AI and machine learning? We are always looking for innovative thinkers and practitioners to join our team! Here’s what you can expect when you start a career with us:
- **Dynamic Work Environment**: Work in an atmosphere that thrives on creativity and forward-thinking.
- **Professional Growth**: Opportunities to learn and advance your career while making a significant impact on the AI community.
- **Remote and Flexible Work**: We offer flexible working options to fit your lifestyle.

### Get Involved

Join us in building the future of AI. Whether you are a developer, researcher, or simply an enthusiast, there are numerous ways to engage with the Hugging Face community:
- **Explore AI Apps**: Discover and utilize applications that showcase our community’s innovation.
- **Contribute Models and Datasets**: Become a part of our extensive library by sharing your work.
- **Participate in Discussions**: Connect with like-minded individuals in our forums and social media channels.

### Contact Us

Interested in learning more? Visit our website to explore our models, datasets, and community resources. Together, we can shape the future of AI.

**Website**: [Hugging Face](https://huggingface.co)  
**Follow Us** on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/hugging-face)

---

Join us at Hugging Face, where innovation meets collaboration in artificial intelligence!

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face! 🤗

## The AI Community Building the Future! 🌟

Are you ready to become part of something BIG? Think of Hugging Face as an artificial intelligence party, where everyone is invited (no awkward small talk required).

---

### What We Do! 🎉

At Hugging Face, we’re on a mission to build the future of AI. Imagine a world where algorithms hug every problem away—well, kind of! Here’s a sneak peek at our playground:

- **Models Galore!** 🚀  
  With over **1 million models** to choose from, we’ve got what you need. Need an algorithm that can cook you dinner? We’re working on it!

- **Datasets Delight!** 📊  
  Browse through **250k+ datasets**. Don't worry! They're way less messy than your average kitchen!

- **Spaces of Innovation!** ✨  
  Let your creativity run wild in our *Spaces*—think of it as the ultimate art studio for AI. 

---

### Our Customers! 👩‍💻👨‍💻

We’re proud to be the go-to platform for over **50,000 organizations**, including:

- Amazon (still can't get those Prime packages delivered by drones though!)
- Google (They say "Don't be evil!" but we say "Just be HUGGING!")
- Microsoft (Who knows, maybe they’ll finally get Clippy a new job!)

Join the ranks of brilliant innovators who have already realized the AI future starts here! ✅

---

### Company Culture! 🥳

At Hugging Face, our culture is all about collaboration—like a potluck, but instead of dishes, we bring cutting-edge AI models!

- **Team Spirit:** With our team of 210+ genius-level nerds 📐🧠, expect a vibrant mix of brains and humor! 

- **Open Source Love:** Sharing is caring! We believe in open-source because who doesn’t like free stuff? Especially if it comes with a side of collaboration!

- **Flexibility:** We understand that not every superstar works well in an office cubicle. Work from anywhere you like—pajamas encouraged!

---

### Careers! 🌈

Looking for a job where you can flex your brain muscles and enjoy excellent workplace banter? 

### Join Us!  
If you're passionate about machine learning and are ready to help build the future, check our careers page! You might just land a dream job as a **Model Whisperer** or **Dataset Ninja**! 🤖🥷

- **Perks Include:**  
  - Remote Work Opportunities 🏠  
  - Competitive Salaries 💰  
  - Endless Coffee and Snacks ☕🍩

---

### Ready to Hug It Out? 🤗

Whether you’re a prospective customer, investor, or looking to join the crew, come visit us at [Hugging Face](https://huggingface.co). Let's shape the AI future together—with a hug! 

---

**Disclaimer:** No models were harmed in the making of this brochure. Please keep shoulders from grinding together during any brainstorming sessions!

In [26]:
# Doing it now with the snarky version:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}



# Welcome to Hugging Face: The AI Community Building the Future (or is it?)

---

## Who Are We? 🤔

Ah, *Hugging Face*—because what better way to describe Artificial Intelligence than with a heartwarming embrace? This corporate wonderland is where the machine learning community pretends to collaborate on groundbreaking models, datasets, and applications. So let’s dive into what we do best: **Buzzword Bingo**.

---

## What We Offer 🎁

### **Models, Models, and Did We Mention Models?**
1M+ models ready to choose from, because who needs originality when you can have choices? Our models are updated so frequently, they deserve a participation trophy for simply existing.

### **Datasets Galore! 📊**
We have a staggering collection of datasets (250k!) that will make you wonder, “Why did I even need a personal life?” Dive deep into the mathematical abyss while others are out enjoying sunny days.

### **Spaces (Not Just for Astronauts) 🚀**
Experience unparalleled creativity in our *Spaces*, running on GPUs named after cartoon heroes (we assume). Enhance your work-life balance by optimizing those rendering times instead of optimizing your social life!

---

## Join the In-Crowd (Or Else) 👥

“More than 50,000 organizations are using Hugging Face.” That’s right, *more than 50,000*! Who wants exclusivity? Don’t worry, we’re basically the Costco of the AI world—lots of stuff, zero privacy.

---

## Work-Life Balance Initiatives 🏖️

Get ready for corporate-led buzzword initiatives that would make even the most optimistic team lead roll their eyes. Enjoy cutting-edge work-life balance solutions like *mandatory* overtime, and the *daily* Slack “How’s everyone doing?” check-ins—perfect for reminding you that you are, indeed, *still* at work.

### **Career Opportunities🌟**
Sign up to curate AI models by day and drown in spreadsheets by night! Experience career growth opportunities where “you can’t leave until the project is finished” translates into “just a little more overtime.”

---

## Why Choose Us? 🎓

- **Transformers**? We’ve got 141,673 versions, each one promising to change the world—anytime now.
- **Diffusers**? Oh, we have 28,159 of those too! Because why leave the world unsaturated when you can apply that AI-sauce everywhere?
- Lots of *Python* libraries (because hope is not a strategy, but reading code might be).

---

## Contact Us 📨

Slide into our DMs through GitHub, Twitter, or LinkedIn! We’re just a few buzzwords away from responding, though if you’re feeling bold, you can try the *HuggingChat* to receive robotic empathy.

---

So, what are you waiting for? Join *Hugging Face* today, the community where *everyone* is building the future—while you're busy trying to figure out what “neural network optimization” actually means in layman's terms. Don't forget to hug your colleagues!



In [30]:
stream_brochure("Atlassian", "https://www.atlassian.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.atlassian.com/company'}, {'type': 'careers page', 'url': 'https://www.atlassian.com/company/careers'}, {'type': 'events page', 'url': 'https://www.atlassian.com/company/events'}, {'type': 'blog page', 'url': 'https://www.atlassian.com/blog'}, {'type': 'investor relations page', 'url': 'https://investors.atlassian.com'}, {'type': 'foundation page', 'url': 'https://www.atlassianfoundation.org/'}]}



# Welcome to Atlassian: Where Collaboration Meets Chaos!

## What We Do
Atlassian is the proud parent of an *exhaustive* suite of collaboration tools designed for software, IT, and business teams. Because what’s better than a collaborative software project? Answer: forty-seven collaborative software projects. Just in case one of them actually works out.

### Our Magnificent Products
- **Jira**: Flexible project management, perfect for managing 99 problems but also creating a few of your own.
- **Confluence**: All your knowledge in one place (unless you misplace the password).
- **Jira Service Management**: High-velocity service delivery, for when your services need to be a little *too* high-velocity.
- **Rovo & Others**: Unlock enterprise knowledge while we lock you in endless meetings.

### The New & Shiny
- **Jira Product Discovery**: Because if you capture and prioritize *one more idea*, the world might just explode with ingenuity.
- **Compass**: Optimizing software health—because let’s face it; software is basically a toddler with a cold.
- **Loom**: Quick video updates to replace the real-life conversations you were too busy to have.

## Why Choose Atlassian? 
Sure, every company says they care. But at Atlassian, we *super care*! We have integrations to connect thousands of apps because if you can’t beat the chaos, you might as well embrace it… and invite it over for tea.

### Customers? Oh, We've Got Those!
Our case studies are tales of teamwork glory… or cautionary tales about the dangers of relying too much on collaboration.

## Culture at Atlassian: A Work-Life *Balance*
Here, enjoy the illusion of work-life balance thanks to our renowned *Culture of Collaboration*. We thrive on long meetings and spontaneous brainstorming sessions during coffee breaks. Because who needs personal time when you can brainstorm another app nobody asked for?

- **Remote Work?** Oh sure! As long as you still log into seventeen different platforms to communicate with the team from the *comfort* of your home office that you should really consider redesigning.
  
## Want to Join Us?
Looking for a fulfilling career? Look no further! From Developers to Product Managers, we've got the perfect post-rat-race role for you.

### Apply Now!
Because if you're not working in a high-velocity service delivery mode with flexible project management tools, are you even living? Fill out the form which we won't read, and may the odds be ever in your favor!

## Final Thoughts
Atlassian: Turning complex collaboration into an art form since… well, it’s unclear when. Join us in our merry quest to manage projects better, faster, and with a sprinkle of delightful chaos. 

### Connect with Us
Visit our website, but honestly, could the real knowledge be in the cloud? There's only one way to find out!



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>